# Topic-Spesific Crawler (Text Only)
## Data Mining and Knowledge Discovery
## Dr. Saed Alqaraleh
## 2020-2021

### By: Yahya Tawil
### May 2021



In [ ]:
!pip install tldextract
!pip install transformers


In [ ]:
import csv
import ssl
import os
import ast
from urllib.request import Request, urlopen, URLError, urljoin
from urllib.parse import urlparse
import time
import threading
import queue
from bs4 import BeautifulSoup
from bs4.element import Comment
from pathlib import Path
import re
from datetime import datetime
from queue import PriorityQueue, Empty
from matplotlib import pyplot as plt
import tldextract
import hashlib
import requests
from google.colab import files as FILE
import pathlib
import pandas as pd

import torch
from transformers import pipeline
from transformers import CLIPProcessor, CLIPModel
from torch.nn import functional as F
from PIL import Image

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
### Script Settings ###
conf = dict(
    dir = '/content/drive/My Drive/KAIST/Spider1/',
    imgsDir = "/content/drive/My Drive/KAIST/Spider1/imgs/",
    base_url = ['https://www.worldhistory.org', # history
                'https://www.newworldencyclopedia.org/', # history
                'https://www.ushistory.org', # history
                'https://www.historic-uk.com/', # history
                'https://hbr.org/', # bussiness
                'https://newpol.org/', # politics
                'https://pc.net/', # technology
                'https://www.computerhope.com/', # technology
                'https://www.computerlanguage.com/'], # technology
    number_of_threads = 9,
    labels = ['technology','business','politics', 'history'],
    blacklist = ['facebook','instagram','youtube'],
    mode = "summarize",
    image_weight = 0.3
)

start_time = int(time.time())
crawled_csv = conf['dir']+'crawler_'+"{}.csv".format(start_time)
###

In [ ]:
speed_samples_10S = [] # number of total download pages
speed_samples_10S_ctr = 0 # counter oh how many 10s sample we have
pages_count = 0

lck = threading.Lock()

#Delete the Crawler Directory to start from a clean dirctory
if os.path.exists(conf['dir']) is False:
  print("Create Directory")
  os.mkdir(conf['dir'])

if os.path.exists(conf['imgsDir']) is False:
  print("Create Imgs Directory")
  os.mkdir(conf['imgsDir'])

crawler_csv_file = open(crawled_csv, mode='a')
crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
crawlerFile_writer.writerow(["timestamp","hash","link","post_text","downloaded_imgs","processed","label","true_label"])
pages_count = pages_count +1
crawler_csv_file.close()

Create Directory
Create Imgs Directory


In [ ]:
class MyPriorityQueue(PriorityQueue):
    def __init__(self):
        PriorityQueue.__init__(self)
        self.counter = 0

    def put(self, item, priority):
        PriorityQueue.put(self, (priority, self.counter, item))
        self.counter += 1

    def get(self, *args, **kwargs):
        _, _, item = PriorityQueue.get(self, *args, **kwargs)
        return item

In [ ]:
class Classifier(threading.Thread):
  def __init__(self):
    threading.Thread.__init__(self)

  def summarize_text(self, text, max_tokens=70):
    """Perform light summarization on the input text."""

    tokenizer = summarizer.tokenizer
    inputs = tokenizer.encode(text, truncation=True, max_length=1000)
    truncated_text = tokenizer.decode(inputs, skip_special_tokens=True)

    summary = summarizer(
        truncated_text,
        max_length=max_tokens,
        min_length=20,
        do_sample=False
    )[0]['summary_text']

    return summary

  def chunk_text(self, text, max_len=77, stride=60):
    """Split long text into overlapping chunks based on tokenizer."""
    encoded = clip_processor(text, return_attention_mask=False, return_token_type_ids=False, add_special_tokens=False)
    input_ids = encoded['input_ids']

    chunks = []
    for i in range(0, len(input_ids), stride):
      chunk_ids = input_ids[i:i + max_len]
      if not chunk_ids:
        continue
      chunk_text = clip_processor.decode(chunk_ids, clean_up_tokenization_spaces=True)
      chunks.append(chunk_text)
      if len(chunk_ids) < max_len:
        break
    return chunks

  def get_label(self, sentence, images=None, image_weight=0.5, mode = None):
    """
    Classify the input sentence (and optional images) against known labels.
    Combines chunked sentence embeddings with image embeddings weighted by image_weight.
    """
    max_len = 77  # max_position_embeddings for CLIP

    if mode == "chunk":
      sentence_chunks = self.chunk_text(sentence)
      texts = sentence_chunks + conf['labels']

      text_inputs = clip_processor(
          text=texts,
          return_tensors="pt",
          padding=True,
          truncation=True,
          max_length=max_len
      )

      with torch.no_grad():
          text_outputs = clip_model.get_text_features(**text_inputs)

      chunk_embs = text_outputs[:len(sentence_chunks)]
      label_embs = text_outputs[len(sentence_chunks):]
      sentence_emb = chunk_embs.mean(dim=0, keepdim=True)

    elif mode == "summarize":
      # Summarization mode
      summarized = self.summarize_text(sentence, max_tokens=60)
      texts = [summarized] + conf['labels']

      text_inputs = clip_processor(
          text=texts,
          return_tensors="pt",
          padding=True,
          truncation=True,
          max_length=max_len
      )

      with torch.no_grad():
          text_outputs = clip_model.get_text_features(**text_inputs)

      sentence_emb = text_outputs[0:1]
      label_embs = text_outputs[1:]

    else:
      texts = [sentence] + conf['labels']
      text_inputs = clip_processor(
        text=texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_len
      )

    if images:  # images present
        image_inputs = clip_processor(
            images=images,
            return_tensors="pt"
        )
        with torch.no_grad():
            image_outputs = clip_model.get_image_features(**image_inputs)
        image_emb = image_outputs.mean(dim=0, keepdim=True)

        combined_emb = (1 - image_weight) * sentence_emb + image_weight * image_emb

    else:
        combined_emb = sentence_emb

    combined_emb = F.normalize(combined_emb, dim=-1)
    label_embs = F.normalize(label_embs, dim=-1)

    similarities = F.cosine_similarity(combined_emb, label_embs)
    closest = similarities.argsort(descending=True)

    label = f"{conf['labels'][closest[0]]}:{similarities[closest[0]]:.3f}," \
                f"{conf['labels'][closest[1]]}:{similarities[closest[1]]:.3f}"
    return label

  def run(self):
    time.sleep(15) # sleep 10 s until the downloader get some data
    while True:
      # reading the csv file
      lck.acquire()
      df = pd.read_csv(crawled_csv,index_col=False)
      df['label'] = df['label'].astype(object)
      for i in range(0,len(df. index)):
        if df.loc[i, 'processed'] == "No":
          df.loc[i, 'processed'] = 'Yes'

          sentence = df.loc[i, 'post_text']
          images_names = df.loc[i, 'downloaded_imgs']
          images_names = ast.literal_eval(images_names)

          images_paths = [conf['imgsDir'] + name for name in images_names]

          valid_images = []
          for path in images_paths:
            try:
              img = Image.open(path).convert("RGB")
              if img.mode == 'P':
                img = img.convert('RGBA')
              else:
                img = img.convert('RGB')
              valid_images.append(img)
            except:
              print(f"Skipped file {path}")

          prediction = self.get_label(sentence, valid_images, conf["image_weight"], conf["mode"])
          df.loc[i, 'label'] = [prediction]

          with open(crawled_csv, 'w') as f:
            df.to_csv(f,index=False)
          lck.release()
          i = i +1
          break
        i = i +1
        if i == len(df. index) - 1:
          time.sleep(10) # sleep 10 sec when there are no more enough data
      time.sleep(0.5)


In [ ]:
regular_express = re.compile(r"https?://(\.)?")

class Crawler(threading.Thread):
    def __init__(self,base_url,url_queue,have_visited, error_links,url_lock,thread_num):

        threading.Thread.__init__(self)
        print(f"Web Crawler worker {threading.current_thread()} has Started")
        self.base_url = base_url
        self.links_queue= url_queue
        self.have_visited = have_visited
        self.error_links = error_links
        self.url_lock = url_lock
        self.threadNo = thread_num

    def tag_visible(self,element): # to exclude the following tags content
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        if re.match(r"[\n]+",str(element)): return False
        return True

    def run(self):
        global lck
        global pages_count
        my_ssl = ssl.create_default_context()
        my_ssl.check_hostname = False
        my_ssl.verify_mode = ssl.CERT_NONE

        while True:
            now = datetime.now()
            timestamp = datetime.timestamp(now)
            #print("Queue Size {} in thread{}:".format(self.links_queue.qsize(),self.threadNo))
            try:
              link = self.links_queue.get()
              link = urljoin(self.base_url,link)
              domain = tldextract.extract(link).domain
              if domain in conf['blacklist']:
                continue

              domain_have_visited = self.have_visited.get(domain,0)
              if domain_have_visited == 0:
                domain_have_visited = []

              # if the link is None the queue is exhausted or the threads are yet
              # process the links.
              if link is None:
                  break

              # if The link is already visited we break the execution unless it passed 24h since last visit.
              if len(domain_have_visited) != 0:
                find=[item for item in domain_have_visited if link in item]
                if len(find)>0:
                  find=tuple(find)
                  if (now - find[0][1]).total_seconds() < 86400: # 86400 is 24H in secs, if the last visit time is less than 24h, then don't visit
                    # print(f"The link {link} is already visited")
                    continue
                  else:
                      self.have_visited[domain].remove(find[0]) #remove the record, to add new one later

              try:
                  req = Request(link, headers= {'User-Agent': 'Mozilla/5.0'})
                  try:
                    response = urlopen(req, context=my_ssl)
                  except:
                    continue

                  soup = BeautifulSoup(response.read(),"html.parser")
                  text=soup.prettify()

                  images = soup.find_all('img')
                  # print(f"Total {len(images)} Image Found!")

                  downloaded_imgs = []
                  hash = int(hashlib.sha256(link.encode('utf-8')).hexdigest(), 16) % 10**8

                  if len(images) != 0:
                    for i, image in enumerate(images):
                      try:
                          # In image tag ,searching for "data-srcset"
                          image_link = image["data-srcset"]

                      # then we will search for "data-src" in img
                      # tag and so on..
                      except:
                          try:
                              # In image tag ,searching for "data-src"
                              image_link = image["data-src"]
                          except:
                              try:
                                  # In image tag ,searching for "data-fallback-src"
                                  image_link = image["data-fallback-src"]
                              except:
                                  try:
                                      # In image tag ,searching for "src"
                                      image_link = image["src"]
                                  # if no Source URL found
                                  except:
                                      pass

                      try:
                        r = requests.get(image_link).content
                        # After checking above condition, Image Download start
                        extention = pathlib.Path(image_link).suffix
                        if extention == ".png" or extention == ".jpg" :
                          with open(f"{conf['imgsDir']}{hash}_{i}{extention}", "wb") as f:
                              downloaded_imgs.append(f"{hash}_{i}{extention}")
                              # print("img saved!")
                              f.write(r)
                              f.close()
                        else:
                          pass
                      except:
                        pass
                  texts = soup.find_all(string=True)

                  visible_texts = filter(self.tag_visible, texts)
                  text = u",".join(t.strip() for t in visible_texts)
                  text = text.lstrip().rstrip()
                  text = text.split(',')
                  post_text = ''
                  for sen in text:
                    if sen:
                      if len(sen.split(' ')) > 5: # Accept only texts with more than 5 words
                        sen = sen.rstrip().lstrip()
                        post_text += sen+' '
                  if len(post_text.split(' ')) < 6:
                    continue
                  lck.acquire()
                  crawler_csv_file = open(crawled_csv, mode='a')
                  crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                  crawlerFile_writer.writerow([timestamp,hash,link,post_text,downloaded_imgs,"No","N/A"," "])
                  pages_count = pages_count +1
                  crawler_csv_file.close()
                  lck.release()

                  if type(soup.find_all('a')) == 'NoneType':
                    print("No links found in this page!")
                    continue
                  else:
                    for a_tag in soup.find_all('a'):
                        links = a_tag.get("href")
                        if len(domain_have_visited) == 0:
                            self.links_queue.put(links,1)
                        else:
                            if (links not in domain_have_visited) :
                              # if (links.find('.html#') == -1): # do not crawl the intra document links
                              self.links_queue.put(links,1)

                  # print(f"Adding {link} to the crawled list at {timestamp}")
                  if len(domain_have_visited) != 0:
                    self.have_visited[domain].append((link,now))
                  else:
                    self.have_visited[domain] = [(link,now)]
              except URLError as e:
                  #print(f"URL {link} threw this error {e.reason} while trying to parse")
                  self.error_links.append(link)
            finally:
                self.links_queue.task_done()
        print("End of thread {}".format(self.threadNo))

In [ ]:
def count_downloaded_pages():
    global speed_samples_10S_ctr
    threading.Timer(10, count_downloaded_pages).start()
    speed_samples_10S.append(pages_count)
    speed_samples_10S_ctr = speed_samples_10S_ctr + 1

    print("{} total pages downloaded till now & {} threads are active, {} x10 sec".format(pages_count,threading.active_count(),speed_samples_10S_ctr))

    #Each one hour of execution plot the number of download webpages
    if speed_samples_10S_ctr > 360:
      speed_samples_10S_ctr = 0
      plt.plot(speed_samples_10S)
      plt.show()

    for i,thread in enumerate(crawler_threads):
      if thread.is_alive() == False and thread.name == "crawler":
      #   print("Thread No. {} is alive".format(i))
      # else:
        # print("Thread No. {} is dead ".format(i))
        crawler = Crawler(base_url = thread.base_url,
                  url_queue = thread.links_queue,
                  have_visited= thread.have_visited,
                  error_links= thread.error_links,
                  url_lock=thread.url_lock,
                  thread_num = thread.threadNo)
        del crawler_threads[i] #remove the dead thread
        crawler.start()
        crawler_threads.append(crawler)

        crawler.join()
      elif thread.is_alive() == False and thread.name == "classifer":
          classifier = Classifier()
          classifier.name="classifer"
          del crawler_threads[i] #remove the dead thread
          classifier.start()
          crawler_threads.append(classifier)


In [ ]:
def monitor_thread():
    global speed_samples_10S_ctr
    while True:
        time.sleep(10)

        speed_samples_10S.append(pages_count)
        speed_samples_10S_ctr += 1

        print(f"{pages_count} total pages downloaded, {threading.active_count()} threads active, {speed_samples_10S_ctr} x10 sec")

        if speed_samples_10S_ctr > 360:
            speed_samples_10S_ctr = 0
            plt.plot(speed_samples_10S)
            plt.show()

        for i, thread in enumerate(crawler_threads[:]):  # copy to avoid modifying during iteration
            if not thread.is_alive():
              crawler_threads.remove(thread)


# Main

In [ ]:
print("The Crawler has the following configuraions:")
print(conf)

base_url = conf["base_url"]
number_of_threads = conf["number_of_threads"]

url_lock = threading.Lock()


have_visited = []
crawler_threads = []
error_links = []

count_downloaded_pages()

monitor = threading.Thread(target=monitor_thread)
monitor.start()

before_starting = datetime.now()

links_to_crawl=[]

for i in range(int(number_of_threads)):
    print("Setup thread no.{} with the following settings:\nbase_url:{}".format(i,base_url[i]))
    links_to_crawl.append(MyPriorityQueue())
    links_to_crawl[i].put(base_url[i],1)
    have_visited.append({})
    error_links.append([])
    crawler = Crawler(base_url = base_url[i],
                      url_queue = links_to_crawl[i],
                      have_visited= have_visited[i],
                      error_links= error_links[i],
                      url_lock=url_lock,
                      thread_num = i)
    crawler.name = "crawler"
    classifier = Classifier()

    crawler.start()
    classifier.name="classifer"
    classifier.start()
    crawler_threads.append(crawler)
    crawler_threads.append(classifier)
for crawler in crawler_threads:
    crawler.join()

The Crawler has the following configuraions:
{'dir': '/content/drive/My Drive/KAIST/Spider1/', 'imgsDir': '/content/drive/My Drive/KAIST/Spider1/imgs/', 'base_url': ['https://www.worldhistory.org', 'https://www.newworldencyclopedia.org/', 'https://www.ushistory.org', 'https://www.historic-uk.com/', 'https://hbr.org/', 'https://newpol.org/', 'https://pc.net/', 'https://www.computerhope.com/', 'https://www.computerlanguage.com/'], 'number_of_threads': 9, 'labels': ['technology', 'business', 'politics', 'history'], 'blacklist': ['facebook', 'instagram', 'youtube']}
Setup thread no.0 with the following settings:
base_url:https://www.worldhistory.org
Web Crawler worker <_MainThread(MainThread, started 135702450438784)> has Started
Setup thread no.1 with the following settings:
base_url:https://www.newworldencyclopedia.org/
Web Crawler worker <_MainThread(MainThread, started 135702450438784)> has Started
Setup thread no.2 with the following settings:
base_url:https://www.ushistory.org
Web Cr

117 total pages downloaded, 23 threads active, 1 x10 sec


151 total pages downloaded, 23 threads active, 2 x10 sec
151 total pages downloaded, 23 threads active, 3 x10 sec


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 60, but your input_length is only 51. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)
Your max_length is set to 60, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


158 total pages downloaded, 23 threads active, 4 x10 sec


...

Your max_length is set to 60, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/45318705_1.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/45318705_2.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/45318705_3.png
558 total pages downloaded, 23 threads active, 65 x10 sec
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/96700347_1.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/96700347_2.png
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/108378_0.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/108378_1.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/108378_2.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/108378_3.jpg
562 total pages downloaded, 23 threads active, 66 x10 sec
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/34869910_0.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/34869910_1.jpg


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


567 total pages downloaded, 23 threads active, 67 x10 sec
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_0.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_1.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_2.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_3.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_4.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/72290067_5.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_1.png
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_3.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_4.png
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_5.png
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_6.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_7.jpg
Skipped file /content/drive/My Drive/KAIST/Spider1/imgs/24166602_8.jpg
Skipped file /conte

KeyboardInterrupt: 